### LangChain - Database Agents

In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

In [2]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo"
temperature = 0.1
max_tokens = 2000

llm = ChatOpenAI(
    model = model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key = openai_api_key
)

In [3]:
db_file_path = "./chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

In [ ]:
query = "SELECT * FROM Artist LIMIT 10;"

db.run(query)

In [ ]:
from langchain.chains import create_sql_query_chain

In [ ]:
chain = create_sql_query_chain(llm, db)

response = chain.invoke({
    "question": "how many employees are working in the organization?"
})

In [ ]:
response

In [ ]:
db.run(response)

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.tools import tool

@tool
def parse(query_string: str) -> str:
    """
    Parses Generated SQL Statement
    """
    
    splitted = query_string.split(":")
    
    if(len(splitted) >= 2):
        query = splitted[1]
    else:
        query = query_string
    
    return query
  

In [ ]:
query = """
QUERY: SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

print(parse(query))


In [ ]:
query = """
SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

print(parse(query))


In [ ]:
execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)

chain = write_query | parse | execute_query

In [ ]:
response = chain.invoke({
    "question": "How many employees are working in the organization currently?"
})

print(response)


In [ ]:
# response = chain.invoke({
#     "question": "how many species of animals are there in the zoo?"
# })

# print(response)


In [ ]:
# chain = create_sql_query_chain(llm, db)

# response = chain.invoke({
#     "question": "how many species of animals are there in the zoo?"
# })

# response

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL query, and SQL result, answer the user question.
    
    Question: {question}
    SQL Query: {query}
    Result: {result}
    Answer:
    """
)

In [ ]:
new_chain = RunnablePassthrough.assign(query = write_query) \
        .assign(result = itemgetter("query") | parse | execute_query) | \
        answer_prompt | llm | StrOutputParser()

In [ ]:
question = "How many employees are working in the organization currently?"

response = new_chain.invoke({
    "question": question
})

response


In [ ]:
chain = create_sql_query_chain(llm, db)

response = chain.invoke({
    "question": "which country's customers have spent the most?"
})

response

In [ ]:
question = "which country's customers have spent the most?"

response = new_chain.invoke({
    "question": question
})

print(response)

In [4]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

In [ ]:
tools

In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

SQL_PREFIX="""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""

system_message = SystemMessage(content=SQL_PREFIX)

agent_executor = create_react_agent(
    llm,
    tools = tools,
    state_modifier = system_message
)

In [6]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Which country's customers have spent the most?")
    ]
}):
    print(stream)
    print("***********")
    

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8ppsWYohs1E1D3t5gDuA8mFd', 'function': {'arguments': '{"query":"SELECT c.Country, SUM(i.Total) AS Total_Spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_Spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 559, 'total_tokens': 612, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoWKJChWbCm8BX5Vqw8GxabInGwTk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9d181847-7d48-4fff-8f99-7a73b9fefc31-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT 

In [7]:
agent_executor.invoke({
    "messages": [
        HumanMessage(content="Which country's customers have spent the most?")
    ]
})

{'messages': [HumanMessage(content="Which country's customers have spent the most?", additional_kwargs={}, response_metadata={}, id='c7967b6b-d036-4184-b0fd-6b2baa8cb5f0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qHqHEoDq6w3ZcZS9YSGpQVIO', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 559, 'total_tokens': 571, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoWLBipVwkApl6fcIyz18W7syMTLv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e3ea679c-4a2d-486c-8ee0-dd84a1aa3423-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_qHqHEoDq6w3